### Set up JADX

Install jadx if not exist

In [1]:
%%sh

if [[ "$(which jadx)" == "" ]]; then
    if [[ "$OSTYPE" == "darwin"* ]]; then
        brew install -y jadx
    else
        sudo pacman -S jadx
    fi
else
    echo "jadx path: $(which jadx)"
    echo "version: $(jadx --version)"
fi

jadx path: /opt/homebrew/bin/jadx
version: 1.3.3


### Decompile the APK in source directory with JADX

With configured `ROOT_PATH`, execute JADX on each presented APK files (with .apk extension)

In [41]:
%%sh

ROOT_PATH='/Users/funnyfeb/research/thesis/fire-in-the-hole'

PATH_TO_APK="$ROOT_PATH/apk"
PATH_TO_DECOMPILE="$ROOT_PATH/decompiled"
PATH_TO_RAWDATA="$ROOT_PATH/rawdata"

rm -Rf $PATH_TO_DECOMPILE
rm -Rf $PATH_TO_RAWDATA

cd $PATH_TO_APK

for f in *.apk; do
    echo ":: Start decompiling $f ------------"
    mkdir -p "$PATH_TO_RAWDATA/$f"
    mkdir -p "$PATH_TO_DECOMPILE/$f"
    jadx \
      -ds "$PATH_TO_DECOMPILE/$f/source" \
      -dr "$PATH_TO_DECOMPILE/$f/resource" \
      --deobf -v \
      $(echo $f) > "$PATH_TO_RAWDATA/$f/jadx-decompile.log"
    echo ":: End decompiling $f --------------"
done


:: Start decompiling 7-Eleven TH_11.39.0_apkcombo.com.apk ------------
:: End decompiling 7-Eleven TH_11.39.0_apkcombo.com.apk --------------
:: Start decompiling AntennaPod_2.5.0_apkcombo.com.apk ------------
:: End decompiling AntennaPod_2.5.0_apkcombo.com.apk --------------
:: Start decompiling MorChana - หมอชนะ_2.1.5_apkcombo.com.apk ------------
:: End decompiling MorChana - หมอชนะ_2.1.5_apkcombo.com.apk --------------
:: Start decompiling SellConnect_2.3.1_apkcombo.com.apk ------------
:: End decompiling SellConnect_2.3.1_apkcombo.com.apk --------------
:: Start decompiling com.facebook.katana_358.0.0.5.117-311612936_minAPI29(arm64-v8a)(240,213dpi)_apkmirror.com.apk ------------
:: End decompiling com.facebook.katana_358.0.0.5.117-311612936_minAPI29(arm64-v8a)(240,213dpi)_apkmirror.com.apk --------------
:: Start decompiling com.kasikorn.retail.mbanking.wap-5.3.3-free-www.kekaku.com.apk ------------
:: End decompiling com.kasikorn.retail.mbanking.wap-5.3.3-free-www.kekaku.com.apk

### Extract the import statement from decompiled code



In [56]:
%%sh

ROOT_PATH='/Users/funnyfeb/research/thesis/fire-in-the-hole'

PATH_TO_DECOMPILE="$ROOT_PATH/decompiled"
PATH_TO_RAWDATA="$ROOT_PATH/rawdata"

cd $PATH_TO_DECOMPILE

for f in *; do
    grep -R -e "^import \S*;$" $(echo "$f/source/*") > $(echo "$PATH_TO_RAWDATA/$f/import_list.data")
done

grep: 7-Eleven: No such file or directory
grep: TH_11.39.0_apkcombo.com.apk/source/*: No such file or directory
grep: MorChana: No such file or directory
grep: หมอชนะ_2.1.5_apkcombo.com.apk/source/*: No such file or directory


### Extract the GREP result
---

####  `import` statement

With GREP result as following example:

```
com.microsoft.office.word_16.0.14931.20096-2002855229_minAPI26(armeabi-v7a)(nodpi)_apkmirror.com.apk//source/android/content/p006pm/IPackageStatsObserver.java:import android.os.Binder;

import_statement -> import android.os.Binder;
package -> android.os.Binder
```

#### `package` path extraction

With package path from import statement as following

```
java.utils.Map
android.os.Binder
org.apache.http.entity.AbstractHttpEntity
com.facebook.common.dextricks.DexLibLoader
org.apache.http.client.params.HttpClientParams
com.facebook.react.views.picker.ReactDropdownPickerManager
com.google.firebase.components.C4875q
```

##### Approach#1 - Extract group id (first two blocks between first punctuation sign) and artifact (one block after the second punctuation sign)

```
RegEx: (?P<group>\w*\.\w*)\.(?P<artifact>\w*)[|\.]{0,1}

{java.utils, Map}       java.utils.Map
{android.os, Binder}    android.os.Binder
{org.apache, http}      org.apache.http.entity.AbstractHttpEntity
{com.facebook, common}  com.facebook.common.dextricks.DexLibLoader
{org.apache, http}      org.apache.http.client.params.HttpClientParams
{com.facebook, react}   com.facebook.react.views.picker.ReactDropdownPickerManager
{com.google, firebase}  com.google.firebase.components.C4875q
```


#### Output CSV

Filename and importing package are extracted with RegEx and pipe into new CSV file with following fields.

| field            | description 
|:-------          |:-------------
| grep_text        | Raw result from GREP command
| apk_name         | APK file name
| file_name        | JAVA code file name with import statement
| import_statement | Raw statement result (Prefix with `import` keyword)
| package          | Importing package id
| group            | Suspect group id of package owner
| artifiact        | Suspect artifact id of package

In [57]:
import os
import pandas as pd

PATH_TO_RAWDATA = '/Users/funnyfeb/research/thesis/fire-in-the-hole/rawdata/'
IMPORT_LIST_FILE_NAME = 'import_list.data'
IMPORT_EXTRACTED_FILE_NAME = 'import_extracted.data'

IMPORT_TEMPLATE_REGEX = r'\/source\/(?P<file_name>\S*):(?P<import_statement>import \S*)'
PACKAGE_PATH_REGEX = r'import (\S*);'
PACKAGE_ATTR_REGEX = r'(?P<group>\w*\.\w*)\.(?P<artifact>\w*)[|\.]{0,1}'

for apk_name in os.listdir(PATH_TO_RAWDATA):
    data_file_path = os.path.join(PATH_TO_RAWDATA, apk_name, IMPORT_LIST_FILE_NAME)
    target_file_path = os.path.join(PATH_TO_RAWDATA, apk_name, IMPORT_EXTRACTED_FILE_NAME)

    import_df = pd.read_csv(data_file_path, sep='&', header=None, delimiter=None, names=['grep_text'])
    extracted_df = import_df['grep_text'].str.extract(IMPORT_TEMPLATE_REGEX, expand=False)
    import_df['apk_name'] = apk_name
    import_df['file_name'] = extracted_df['file_name']
    import_df['import_statement'] = extracted_df['import_statement']
    import_df['package'] = extracted_df['import_statement'].str.extract(PACKAGE_PATH_REGEX)

    package_df = import_df['package'].str.extract(PACKAGE_ATTR_REGEX, expand=False)
    import_df['group'] = package_df['group']
    import_df['artifact'] = package_df['artifact']

    import_df.to_csv(target_file_path, header=True, index=False)

In [58]:
### Sample
import_df.sample(20)

,grep_text,apk_name,file_name,import_statement,package,group,artifact
9960,com.onedaycat.sellconnect.219.apk/source/com/g...,com.onedaycat.sellconnect.219.apk,com/google/android/material/appbar/CollapsingT...,import androidx.appcompat.widget.Toolbar;,androidx.appcompat.widget.Toolbar,androidx.appcompat,widget
21468,com.onedaycat.sellconnect.219.apk/source/io/in...,com.onedaycat.sellconnect.219.apk,io/intercom/android/sdk/helpcenter/api/HelpCen...,import p339l.C7772q;,p339l.C7772q,NaN,NaN
35584,com.onedaycat.sellconnect.219.apk/source/p056f...,com.onedaycat.sellconnect.219.apk,p056f/p142i/p154q/C5044j.java,import android.os.Build;,android.os.Build,android.os,Build
14861,com.onedaycat.sellconnect.219.apk/source/com/a...,com.onedaycat.sellconnect.219.apk,com/amazonaws/services/securitytoken/model/tra...,import com.amazonaws.services.securitytoken.mo...,com.amazonaws.services.securitytoken.model.Ass...,com.amazonaws,services
19956,com.onedaycat.sellconnect.219.apk/source/io/in...,com.onedaycat.sellconnect.219.apk,io/intercom/android/sdk/blocks/NetworkImage.java,import io.intercom.android.sdk.views.Resizable...,io.intercom.android.sdk.views.ResizableImageView,io.intercom,android
19725,com.onedaycat.sellconnect.219.apk/source/com/p...,com.onedaycat.sellconnect.219.apk,com/pichillilorenzo/flutter_inappwebview/chrom...,import android.graphics.Color;,android.graphics.Color,android.graphics,Color
32791,com.onedaycat.sellconnect.219.apk/source/p056f...,com.onedaycat.sellconnect.219.apk,p056f/p074f/p084c/C3505d.java,import p056f.p074f.p077b.p080r.AbstractC3350c0;,p056f.p074f.p077b.p080r.AbstractC3350c0,p056f.p074f,p077b
5169,com.onedaycat.sellconnect.219.apk/source/coil/...,com.onedaycat.sellconnect.219.apk,coil/util/C1183e.java,import androidx.lifecycle.AbstractC0844i;,androidx.lifecycle.AbstractC0844i,androidx.lifecycle,AbstractC0844i
1383,com.onedaycat.sellconnect.219.apk/source/andro...,com.onedaycat.sellconnect.219.apk,androidx/compose/p005ui/platform/AndroidCompos...,import p056f.p074f.p101e.p110p.p113c.C4262p;,p056f.p074f.p101e.p110p.p113c.C4262p,p056f.p074f,p101e
8084,com.onedaycat.sellconnect.219.apk/source/com/g...,com.onedaycat.sellconnect.219.apk,com/google/firebase/messaging/C2631j0.java,import p198h.p205d.p206a.p207a.AbstractC5473c;,p198h.p205d.p206a.p207a.AbstractC5473c,p198h.p205d,p206a


In [59]:
dedup_df = import_df.drop_duplicates(subset=['apk_name', 'group', 'artifact'])[['apk_name', 'group', 'artifact']]
filtered_df = dedup_df[ dedup_df['group'].str.contains('android.|java.|kotlin.') == False ]

filtered_df.sample(60)

,apk_name,group,artifact
40984,com.onedaycat.sellconnect.219.apk,p375o.p376m0,p387l
230,com.onedaycat.sellconnect.219.apk,p056f.p073e,C3095b
19108,com.onedaycat.sellconnect.219.apk,p056f.p182w,C5254e
5211,com.onedaycat.sellconnect.219.apk,p184g.p197t,AbstractC5443d
2455,com.onedaycat.sellconnect.219.apk,p056f.p073e,C3098d
27464,com.onedaycat.sellconnect.219.apk,io.sentry,SpanId
23670,com.onedaycat.sellconnect.219.apk,p056f.p181v,C5248a
19819,com.onedaycat.sellconnect.219.apk,com.intercom,commons
27440,com.onedaycat.sellconnect.219.apk,io.sentry,IUnknownPropertiesConsumer
43306,com.onedaycat.sellconnect.219.apk,p401r.p402a0,AbstractC8466e


### Search for Package artifact via Maven Repository

Search for suspect package with query in following form:
```
[group] [artifact]

e.g.
    'com.airbnb lottie'
    'com.facebook react'
```

#### Response example:
```
GET 'https://search.maven.org/solrsearch/select?q=com.airbnb lottie&rows=20&wt=json'

{
    "responseHeader": {
        "status": 0,
        "QTime": 3,
        "params": {
            "q": "com.airbnb lottie",
            "core": "",
            "defType": "dismax",
            "spellcheck": "true",
            "qf": "text^20 g^5 a^10",
            "indent": "off",
            "fl": "id,g,a,latestVersion,p,ec,repositoryId,text,timestamp,versionCount",
            "start": "",
            "sort": "score desc,timestamp desc,g asc,a asc",
            "spellcheck.count": "5",
            "rows": "20",
            "wt": "json",
            "version": "2.2"
        }
    },
    "response": {
        "numFound": 2,
        "start": 0,
        "docs": [
            {
                "id": "com.airbnb.android:lottie",
                "g": "com.airbnb.android",
                "a": "lottie",
                "latestVersion": "5.0.3",
                "repositoryId": "central",
                "p": "aar",
                "timestamp": 1646680023000,
                "versionCount": 80,
                "text": [
                    "com.airbnb.android",
                    "lottie",
                    "-sources.jar.sha256",
                    "-javadoc.jar",
                    ".aar.sha256",
                    "-javadoc.jar.sha512",
                    ".aar.sha512",
                    ".module.asc.sha256",
                    ".module.asc.sha512",
                    ".aar.asc.sha512",
                    "-javadoc.jar.sha256",
                    ".module",
                    ".pom.sha512",
                    "-sources.jar",
                    "-sources.jar.asc.sha512",
                    ".aar",
                    ".module.sha256",
                    ".aar.asc.sha256",
                    ".pom",
                    ".module.sha512",
                    "-sources.jar.asc.sha256",
                    "-javadoc.jar.asc.sha256",
                    ".pom.asc.sha256",
                    "-javadoc.jar.asc.sha512",
                    ".pom.asc.sha512",
                    "-sources.jar.sha512",
                    ".pom.sha256"
                ],
                "ec": [
                    "-sources.jar.sha256",
                    "-javadoc.jar",
                    ".aar.sha256",
                    "-javadoc.jar.sha512",
                    ".aar.sha512",
                    ".module.asc.sha256",
                    ".module.asc.sha512",
                    ".aar.asc.sha512",
                    "-javadoc.jar.sha256",
                    ".module",
                    ".pom.sha512",
                    "-sources.jar",
                    "-sources.jar.asc.sha512",
                    ".aar",
                    ".module.sha256",
                    ".aar.asc.sha256",
                    ".pom",
                    ".module.sha512",
                    "-sources.jar.asc.sha256",
                    "-javadoc.jar.asc.sha256",
                    ".pom.asc.sha256",
                    "-javadoc.jar.asc.sha512",
                    ".pom.asc.sha512",
                    "-sources.jar.sha512",
                    ".pom.sha256"
                ]
            },
            {
                "id": "com.airbnb.android:lottie-compose",
                "g": "com.airbnb.android",
                "a": "lottie-compose",
                "latestVersion": "5.0.3",
                "repositoryId": "central",
                "p": "aar",
                "timestamp": 1646680039000,
                "versionCount": 19,
                "text": [
                    "com.airbnb.android",
                    "lottie-compose",
                    "-sources.jar.sha256",
                    "-javadoc.jar",
                    ".aar.sha256",
                    ".aar.sha512",
                    "-javadoc.jar.sha512",
                    ".module.asc.sha256",
                    ".module.asc.sha512",
                    ".aar.asc.sha512",
                    "-javadoc.jar.sha256",
                    "-sources.jar.asc.sha512",
                    "-sources.jar",
                    ".pom.sha512",
                    ".module",
                    ".module.sha256",
                    ".aar",
                    ".aar.asc.sha256",
                    ".pom",
                    "-sources.jar.asc.sha256",
                    ".module.sha512",
                    "-javadoc.jar.asc.sha256",
                    "-javadoc.jar.asc.sha512",
                    ".pom.asc.sha256",
                    ".pom.asc.sha512",
                    ".pom.sha256",
                    "-sources.jar.sha512"
                ],
                "ec": [
                    "-sources.jar.sha256",
                    "-javadoc.jar",
                    ".aar.sha256",
                    ".aar.sha512",
                    "-javadoc.jar.sha512",
                    ".module.asc.sha256",
                    ".module.asc.sha512",
                    ".aar.asc.sha512",
                    "-javadoc.jar.sha256",
                    "-sources.jar.asc.sha512",
                    "-sources.jar",
                    ".pom.sha512",
                    ".module",
                    ".module.sha256",
                    ".aar",
                    ".aar.asc.sha256",
                    ".pom",
                    "-sources.jar.asc.sha256",
                    ".module.sha512",
                    "-javadoc.jar.asc.sha256",
                    "-javadoc.jar.asc.sha512",
                    ".pom.asc.sha256",
                    ".pom.asc.sha512",
                    ".pom.sha256",
                    "-sources.jar.sha512"
                ]
            }
        ]
    },
    "spellcheck": {
        "suggestions": []
    },
    "alternate": [
        "fc:com.airbnb lottie"
    ]
}
```

```
GET 'https://search.maven.org/solrsearch/select?q=com.facebook react&rows=20&wt=json'

{
    "responseHeader": {
        "status": 0,
        "QTime": 2,
        "params": {
            "q": "com.facebook react",
            "core": "",
            "defType": "dismax",
            "spellcheck": "true",
            "qf": "text^20 g^5 a^10",
            "indent": "off",
            "fl": "id,g,a,latestVersion,p,ec,repositoryId,text,timestamp,versionCount",
            "start": "",
            "sort": "score desc,timestamp desc,g asc,a asc",
            "spellcheck.count": "5",
            "rows": "20",
            "wt": "json",
            "version": "2.2"
        }
    },
    "response": {
        "numFound": 3,
        "start": 0,
        "docs": [
            {
                "id": "com.facebook.react:react-native",
                "g": "com.facebook.react",
                "a": "react-native",
                "latestVersion": "0.20.1",
                "repositoryId": "central",
                "p": "aar",
                "timestamp": 1455557396000,
                "versionCount": 17,
                "text": [
                    "com.facebook.react",
                    "react-native",
                    "-sources.jar",
                    "-javadoc.jar",
                    ".aar",
                    ".pom"
                ],
                "ec": [
                    "-sources.jar",
                    "-javadoc.jar",
                    ".aar",
                    ".pom"
                ]
            },
            {
                "id": "io.sariska:com.facebook.react",
                "g": "io.sariska",
                "a": "com.facebook.react",
                "latestVersion": "1.0.0",
                "repositoryId": "central",
                "p": "aar",
                "timestamp": 1618065851000,
                "versionCount": 4,
                "text": [
                    "io.sariska",
                    "com.facebook.react",
                    ".aar.sha512",
                    ".pom.sha256",
                    ".aar",
                    ".aar.sha256",
                    ".pom.sha512",
                    ".pom"
                ],
                "ec": [
                    ".aar.sha512",
                    ".pom.sha256",
                    ".aar",
                    ".aar.sha256",
                    ".pom.sha512",
                    ".pom"
                ]
            },
            {
                "id": "org.webjars.npm:github-com-facebook-react",
                "g": "org.webjars.npm",
                "a": "github-com-facebook-react",
                "latestVersion": "15.6.1",
                "repositoryId": "central",
                "p": "jar",
                "timestamp": 1497608939000,
                "versionCount": 1,
                "text": [
                    "org.webjars.npm",
                    "github-com-facebook-react",
                    "-sources.jar",
                    "-javadoc.jar",
                    ".jar",
                    ".pom"
                ],
                "ec": [
                    "-sources.jar",
                    "-javadoc.jar",
                    ".jar",
                    ".pom"
                ]
            }
        ]
    },
    "spellcheck": {
        "suggestions": []
    },
    "alternate": [
        "fc:com.facebook react"
    ]
}
```

Ref: https://central.sonatype.org/search/rest-api-guide/

In [5]:
import requests
import json

TARGET_HOST = 'https://search.maven.org'
DOMAIN_PATH = '/solrsearch'

def parse_url(uri, query_obj=None):
    query_obj = dict() if query_obj is None else query_obj
    query_obj = urlencode(query_obj)
    query_obj = f"?{query_obj}" if query_obj != "" else ""

    return urljoin(uri, query_obj)



query = {
    'q': 'com.airbnb a:lottie',
    'rows': 20,
    'wt': 'json'
}

host = parse_url('https://search.maven.org/solrsearch/select', query)
x = requests.get(host)

json.loads(x.text)['response']['docs']

[{'id': 'com.airbnb.android:lottie',
  'g': 'com.airbnb.android',
  'a': 'lottie',
  'latestVersion': '5.0.3',
  'repositoryId': 'central',
  'p': 'aar',
  'timestamp': 1646680023000,
  'versionCount': 80,
  'text': ['com.airbnb.android',
   'lottie',
   '-sources.jar.sha256',
   '-javadoc.jar',
   '.aar.sha256',
   '-javadoc.jar.sha512',
   '.aar.sha512',
   '.module.asc.sha256',
   '.module.asc.sha512',
   '.aar.asc.sha512',
   '-javadoc.jar.sha256',
   '.module',
   '.pom.sha512',
   '-sources.jar',
   '-sources.jar.asc.sha512',
   '.aar',
   '.module.sha256',
   '.aar.asc.sha256',
   '.pom',
   '.module.sha512',
   '-sources.jar.asc.sha256',
   '-javadoc.jar.asc.sha256',
   '.pom.asc.sha256',
   '-javadoc.jar.asc.sha512',
   '.pom.asc.sha512',
   '-sources.jar.sha512',
   '.pom.sha256'],
  'ec': ['-sources.jar.sha256',
   '-javadoc.jar',
   '.aar.sha256',
   '-javadoc.jar.sha512',
   '.aar.sha512',
   '.module.asc.sha256',
   '.module.asc.sha512',
   '.aar.asc.sha512',
   '-javadoc